# EXTRACT

## setting variables

In [1]:

import pandas as pd
import numpy as np
import datetime as dt
import logging
import os
import awswrangler as awr

In [2]:

today = pd.Timestamp.today().date()

if today.weekday() == 0:
    yesterday = today - pd.Timedelta(days=3)
else:
    yesterday = today - pd.Timedelta(days=1)

if today.weekday() == 0:
    dbf_yesterday = today - pd.Timedelta(days=4)
else:
    dbf_yesterday = today - pd.Timedelta(days=2)

## extracting

In [3]:
xlsx_anterior = rf"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas\placas_movimentacoes_{yesterday}.xlsx"
xlsx_analise = rf"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas\placas_movimentacoes_{today}.xlsx"

df_anterior = pd.read_excel(xlsx_anterior, engine='openpyxl')
df_analise = pd.read_excel(xlsx_analise, engine='openpyxl')




In [4]:
set_anterior = set(df_anterior)
novos_mask = ~df_analise['chassi'].isin(set(df_anterior['chassi']))
print(f"Quantidade de novos: {novos_mask.sum()}")

Quantidade de novos: 57


In [5]:
set_analise = set(df_analise['chassi'])
desativados_mask = ~df_anterior['chassi'].isin(set_analise)

In [6]:
print(f"Shape de df_analise: {df_analise.shape}")
print(f"Quantidade de desativados: {desativados_mask.sum()}")


Shape de df_analise: (32281, 18)
Quantidade de desativados: 1098


In [7]:
df_desativadas = df_anterior.loc[desativados_mask]
df_desativadas.shape[0]


1098

### RECUPERANDO ATIVOS REMANESCENTES

In [8]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\sql\others_query.sql"
with open(query_path, "r") as file:
    query = file.read()

df_conferencia = awr.athena.read_sql_query(query, database='silver')
df_conferencia.head()  

,placa,chassi,id_placa,id_veiculo,id_carroceria,matricula,conjunto,unidade,consultor,status,...,empresa,coverage,beneficio,categoria,tipo_categoria,data_ativacao_beneficio,status_beneficio,data_atualizacao,consultor_ativo,id_beneficio
0,MRY5651,9BFYCEHV38BB07656,580964,580964,<NA>,3266,4336,TRANSDESK DIGITAL LTDA,Rithieli Ferreira de Lima,NOVO,...,Tag,726259,Assistência 24 Horas,PESADOS - CAMINHÃO,ACIMA DE 15 ANOS,2025-09-29,NOVO,1754-08-30 22:43:41.128654848,S,41
1,MRY5651,9BFYCEHV38BB07656,580964,580964,<NA>,3266,4336,TRANSDESK DIGITAL LTDA,Rithieli Ferreira de Lima,NOVO,...,Tag,726257,Casco PT (COMPLEMENTO),CARROCERIA BAÚ,PADRÃO,2025-09-29,NOVO,1754-08-30 22:43:41.128654848,S,36
2,MRY5651,9BFYCEHV38BB07656,580964,580964,<NA>,3266,4336,TRANSDESK DIGITAL LTDA,Rithieli Ferreira de Lima,NOVO,...,Tag,726256,Assistência a Reparos (VEÍCULO),PESADOS - CAMINHÃO,PADRÃO - 4%,2025-09-29,NOVO,1754-08-30 22:43:41.128654848,S,37
3,MRY5651,9BFYCEHV38BB07656,580964,580964,<NA>,3266,4336,TRANSDESK DIGITAL LTDA,Rithieli Ferreira de Lima,NOVO,...,Tag,726255,Casco PT (VEÍCULO),PESADOS - CAMINHÃO,PADRÃO,2025-09-29,NOVO,1754-08-30 22:43:41.128654848,S,34
4,ELQ5F23,9BVAS02C7BE773053,1246361,1246361,<NA>,3267,4340,UNIDADE CASCAVEL,Cesar Augusto Magalhaes Silva,NOVO,...,Tag,726289,Assistência a Vidros,PESADOS - CAVALO MECÂNICO,DE 1980 ATÉ 2019,2025-09-29,NOVO,1754-08-30 22:43:41.128654848,S,42


In [9]:
df_conferencia_sorted = df_conferencia.sort_values(by='data_ativacao', ascending=False)
df_conferencia_sorted.drop_duplicates(subset='chassi', keep='first', inplace=True)

In [10]:
df_desativadas_status = df_desativadas.merge(
    df_conferencia_sorted[['chassi', 'status_beneficio']],
    on='chassi',
    how='left'
)

In [11]:
df_desativadas_status.shape[0]

1098

In [12]:
df_desativadas_status['status_beneficio_y'].value_counts(dropna=False)

status_beneficio_y
ATIVO        803
CANCELADO    257
NOVO          37
<NA>           1
Name: count, dtype: Int64

In [13]:
df_desativadas_status = df_desativadas_status.drop(columns=['status_beneficio_x'])
df_desativadas_status = df_desativadas_status.rename(columns={'status_beneficio_y': 'status_beneficio'})
df_desativadas_status.columns

Index(['placa', 'chassi', 'id_placa', 'id_veiculo', 'id_carroceria',
       'matricula', 'conjunto', 'unidade', 'consultor', 'cliente',
       'data_registro', 'data_ativacao', 'data_ativacao_beneficio', 'suporte',
       'data_filtro', 'empresa', 'migration_from', 'status_beneficio'],
      dtype='object')

In [14]:
df_desativadas_ativas = df_desativadas_status[df_desativadas_status['status_beneficio'].isin(['ATIVO', 'NOVO'])]
df_desativadas_canceladas = df_desativadas_status[~df_desativadas_status['status_beneficio'].isin(['ATIVO', 'NOVO'])]
df_desativadas_ativas['status_beneficio'] = 'ATIVO'


print(df_desativadas_ativas.shape[0])
print(df_desativadas_canceladas.shape[0])

840
258


C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_23000\312725541.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_desativadas_ativas['status_beneficio'] = 'ATIVO'


In [15]:
df_final = pd.concat([df_analise, df_desativadas_ativas]).drop_duplicates(subset='chassi')

In [16]:
df_final['status_beneficio'].value_counts()

status_beneficio
ATIVO         33092
RENOVAÇÃO        17
MIGRAÇÃO          5
REATIVAÇÃO        3
Name: count, dtype: int64

In [17]:
df_final.head()

,placa,chassi,id_placa,id_veiculo,id_carroceria,matricula,conjunto,unidade,consultor,status_beneficio,cliente,data_registro,data_ativacao,data_ativacao_beneficio,suporte,data_filtro,empresa,migration_from
0,QAW5E74,9BVRG40D2ME886599,1078,1078,0,17532,15890,UNIDADE DOURADOS,Pedro Scheffer,RENOVAÇÃO,ALLAN CHRISTIAN KRUGER,2025-09-22,2025-09-29,2025-09-29,Ana Balicki,2025-09-30,Stcoop,NaN
1,SIV6E52,9536B8TDXPR057552,3709,3709,0,20015,15935,UNIDADE UBERLÂNDIA,Rodrigo Almeida,RENOVAÇÃO,ELEANDRO SINESIO SANTOS XAVIER,2025-09-25,2025-09-29,2025-09-29,João Paulo Ribeiro Pinto,2025-09-30,Stcoop,NaN
2,DSM6E22,9BVRG20C9LE869600,626,626,0,20100,15858,UNIDADE JOINVILLE,Ana Beatriz Prioste,RENOVAÇÃO,EDIR JOSE FIAMONCINI,2025-09-19,2025-09-29,2025-09-29,João Paulo Ribeiro Pinto,2025-09-30,Stcoop,NaN
3,SLJ4J16,9BM951501PB330226,4889,4889,0,20131,15731,UNIDADE CASCAVEL,Aline Dayla Unidades,RENOVAÇÃO,RACOES E CEREAIS NORTE LTDA - ME,2025-09-10,2025-09-29,2025-09-29,João Paulo Ribeiro Pinto,2025-09-30,Stcoop,NaN
4,RXM6A67,96AF1483MNJ004337,1811,0,1811,19750,15894,UNIDADE ITAJAI,Gabriel Soares,RENOVAÇÃO,AGILGROUP LOGISTICA LTDA,2025-09-22,2025-09-29,2025-09-29,João Paulo Ribeiro Pinto,2025-09-30,Stcoop,NaN


In [18]:

lista_chassi_anterior = df_anterior['chassi'].tolist()
novos_mask = ~df_analise['chassi'].isin(lista_chassi_anterior)


In [19]:
df_final.loc[novos_mask, 'status_beneficio'] = 'NOVO'

In [20]:
print(f"Shape de df_anterior: {df_anterior.shape}")
print(f"Shape de df_final: {df_final.shape}")
print(f"Quantidade de novos: {novos_mask.sum()}")

print(f"Quantidade de desativados: {desativados_mask.sum()}")
print(f"Quantidade de desativados ativos: {df_desativadas_ativas.shape[0]}")
print(f"Quantidade de desativados cancelados: {df_desativadas_canceladas.shape[0]}")

Shape de df_anterior: (33318, 18)
Shape de df_final: (33117, 18)
Quantidade de novos: 57
Quantidade de desativados: 1098
Quantidade de desativados ativos: 840
Quantidade de desativados cancelados: 258


# LOAD

In [21]:
xlsx_analise = rf"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas\placas_movimentacoes_{today}.xlsx"
df_final.to_excel(xlsx_analise, engine='openpyxl', index=False)

path_canceladas = r"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas\placas_desativadas.xlsx"
df_desativadas_canceladas.to_excel(path_canceladas, engine='openpyxl', index=False)

xlsx_analise = rf"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\bkp_activation\placas_movimentacoes_{today}.xlsx"
df_final.to_excel(xlsx_analise, engine='openpyxl', index=False)

xlsx_rm_path = rf"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas\placas_movimentacoes_{dbf_yesterday}.xlsx"
if os.path.exists(xlsx_rm_path):
    os.remove(xlsx_rm_path)